# Laboratorio: Recommendation con Algebra Lineare e NumPy

**Programmazione di Applicazioni Data Intensive**  
Laurea in Ingegneria e Scienze Informatiche  
DISI - Università di Bologna, Cesena

Proff. Gianluca Moro, Roberto Pasolini  
nome.cognome@unibo.it

## Recommendation: da strutture dati Python a matrici NumPy

- Nella scorsa esercitazione abbiamo introdotto la **recommendation** di prodotti
  - sappiamo **quali utenti** di un sito di e-commerce **hanno acquistato quali prodotti**
  - vogliamo fornire **suggerimenti personalizzati a ciascun utente** su quali ulteriori prodotti dovrebbe acquistare
  - concentriamo l'analisi su utenti con almeno 30 prodotti distinti acquistati
- Abbiamo implementato una soluzione utilizzando **strutture dati e funzioni standard di Python**
  - dizionari, insiemi, comprehension, ...
- In questa esercitazione ripetiamo gli stessi passaggi usando **array NumPy**
- Vediamo così come un problema reale si possa modellare nell'ambito dell'algebra lineare e risolvere tramite semplici operazioni tra matrici

## Scaricamento File Dati

- Riutilizziamo i dati della scorsa esercitazione, scaricabili all'URL https://git.io/fhxQh
- Se state usando Binder, avete già i file nella directory corrente
- Altrimenti, eseguite la seguente cella di codice per scaricare il file ZIP dall'URL sopra ed estrarne i file

In [1]:
import os.path
if not os.path.exists("purchases_data.zip"):
    from urllib.request import urlretrieve
    urlretrieve("https://git.io/fhxQh", "purchases_data.zip")
    from zipfile import ZipFile
    with ZipFile("purchases_data.zip") as f:
        f.extractall()

## Caricamento Dati

- Ripetiamo i passaggi della scorsa esercitazione per caricare i dati dai file
  - il file `users.csv` contiene ID e nomi degli utenti analizzati: li carichiamo in un dizionario `users` che associ i nomi agli ID
  - facciamo lo stesso col file `items.csv`, che contiene ID e nomi dei prodotti: otteniamo un dizionario `items`
  - il file `purchases-2000.csv` contiene gli acquisti fino alla fine del 2000 in forma di tuple ID utente + ID prodotto: le carichiamo in un insieme `purchases_set`

In [2]:
import csv
with open("users.csv", "r") as f:
    users = {int(uid): name for uid, name in csv.reader(f, delimiter=";")}
with open("items.csv", "r") as f:
    items = {int(iid): name for iid, name in csv.reader(f, delimiter=";")}
with open("purchases-2000.csv", "r") as f:
    purchases_set = {(int(uid), int(iid)) for uid, iid in csv.reader(f, delimiter=";")}

## Algebra Lineare e NumPy

- Oggetto di studio dell'**algebra lineare** sono i vettori, le matrici e le principali operazioni tra di essi
  - con vettori e matrici si possono rappresentare dati da analizzare
  - informazioni d'interesse si possono estrarre con operazioni come il prodotto tra matrici
- **NumPy** è la libreria Python _standard de facto_ per lavorare con array multidimensionali, inclusi vettori e matrici
  - sugli array NumPy si possono compiere efficientemente operazioni elemento per elemento, aggregazioni, operazioni di algebra lineare, ecc.
- Per lavorare con NumPy lo importiamo usando l'alias convenzionale `np`

In [3]:
import numpy as np

## Rappresentare gli Acquisti in forma di Matrice

- L'insieme `purchases` degli acquisti degli utenti analizzati può essere rappresentato in una matrice binaria (ovvero di valori 0 e 1)
  - ogni **riga** corrisponde ad un **utente**
  - ogni **colonna** corrisponde ad un **prodotto** distinto
  - il valore della cella _i,j_ è 1 se l'utente i ha acquistato il prodotto j, 0 altrimenti
- Vediamo come costruire tale matrice

### Assegnazione di Indici a Utenti e Prodotti

- Iniziando definendo **a quali utenti e prodotti corrispondano** rispettivamente **righe e colonne** della matrice
- Costruiamo un dizionario `user_indices` che associ ad ognuno degli N ID utente un numero tra 0 e N-1
  - usiamo `enumerate` per ottenere tuple `(indice, elemento)`, dove _indice_ è un numero progressivo da 0 a N-1
  - usiamo `sorted` per ottenere le chiavi in ordine crescente _(non è strettamente necessario, ma garantisce la piena riproducibilità dei passaggi)_

In [4]:
user_indices = {uid: index for index, uid in enumerate(sorted(users.keys()))}

- Eseguiamo un'operazione simile per ottenere un dizionario `item_indices` con la numerazione degli oggetti

In [5]:
item_indices = {iid: index for index, iid in enumerate(sorted(items.keys()))}

- Abbiamo così ottenuto dizionari che mappano ID utenti e prodotti a righe e colonne della matrice
- Ad esempio, la riga corrispondente all'utente con ID 63776 ha indice...

In [6]:
user_indices[63776]

10

### Inizializzazione della Matrice

- Per allocare la matrice degli acquisti, definiamone dapprima la _forma_, ovvero il numero di righe e di colonne
- Questi sono pari rispettivamente al numero di utenti e di prodotti, che salviamo in due variabili per comodità

In [7]:
n_users = len(users)
n_items = len(items)
print("{} utenti, {} prodotti".format(n_users, n_items))

178 utenti, 3384 prodotti


- Creiamo ora la matrice `purchases` inizializzando tutti i valori a 0, ovvero senza alcun acquisto
- Usiamo per questo la funzione `zeros` passando la forma desiderata della matrice

In [8]:
purchases = np.zeros((n_users, n_items), dtype=np.int32)

### Scrittura Acquisti nella Matrice

- Per ottenere la matrice degli acquisti effettiva, impostiamo ad 1 gli elementi corrispondenti alle coppie utente-oggetto nell'insieme `purchases_set`
- Per ogni tupla, usiamo i dizionari `user_indices` e `item_indices` per individuare la posizione giusta nella matrice e impostiamo ad 1 il valore

In [9]:
for uid, iid in purchases_set:
    i = user_indices[uid]   # riga
    j = item_indices[iid]   # colonna
    purchases[i, j] = 1

- Visualizziamo una porzione della matrice ottenuta...

In [10]:
purchases[-5:, :15]   # ultime 5 righe, prime 15 colonne

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)

- Vediamo ad esempio che l'utente della penultima riga ha acquistato i prodotti di indice 11 e 12
- Ma qual è questo utente e quali sono questi prodotti?

## Associare Nomi a Righe e Colonne

- I dizionari `*_indices` permettono, dato l'ID di un utente o prodotto, di risalire alla riga o colonna corrispondente della matrice
- Come eseguire il passaggio contrario, ovvero dato un indice di riga o colonna risalire all'utente o prodotto?
- Creiamo un vettore `user_names` che contenga i nomi degli utenti nelle posizioni corrispondenti alle righe della matrice
- Allochiamo un vettore vuoto con lunghezza pari al numero di utenti e tipo di elementi `object`, ovvero oggetti Python arbitrari
  - `n_users` (valore singolo) equivale a `(n_users, )` (tupla di un elemento, quindi vettore)

In [11]:
user_names = np.empty(n_users, dtype=object)

- Riempiamo ora il vettore con i nomi degli utenti
  - dal dizionario `users` iteriamo le tuple `(ID, nome)` per ogni utente
  - da `user_indices` otteniamo l'indice della riga corrispondente all'ID utente
  - andiamo a scrivere il nome nella corrispondente posizione in `user_names`

In [12]:
for uid, name in users.items():
    i = user_indices[uid]
    user_names[i] = name

- Eseguiamo una procedura equivalente per ottenere un vettore `item_names` con i nomi dei prodotti

In [13]:
item_names = np.empty(n_items, dtype=object)
for iid, name in items.items():
    item_names[item_indices[iid]] = name

- Possiamo così ottenere il nomi di un singolo utente o prodotto data la riga o colonna

In [14]:
item_names[11]

'Doctor Who - Robot [VHS]'

- In più, rispetto alle liste di Python, possiamo estrarre molteplici valori dal vettore passando una **lista di indici**

In [15]:
item_names[[11, 12]]

array(['Doctor Who - Robot [VHS]', 'Doctor Who - City of Death [VHS]'],
      dtype=object)

- Infine, possiamo usare l'**indicizzazione booleana** per estrarre elementi che corrispondano ad una condizione
- Consideriamo ad esempio la riga 176 della matrice, che indica gli acquisti dell'utente...

In [16]:
user_names[176]

'G.Spider'

- Estraiamo la riga convertendola in un vettore booleano, dove `True` e `False` corrispondono a 1 e 0
  - `[176, :]` si può abbreviare in `[176]`

In [17]:
purchased_by_user_176 = purchases[176, :].astype(bool)
#               oppure: purchases[176, :] == 1
# visualizzo i primi 20 valori
purchased_by_user_176[:20]

array([False, False, False, False, False, False, False, False, False,
       False, False,  True,  True, False, False, False, False, False,
       False, False])

- Possiamo usare questo vettore binario come selettore del vettore `item_names` per i valori corrispondenti a `True`
- In questo modo otteniamo un vettore con i nomi dei soli prodotti acquistati dall'utente

In [18]:
item_names_purchased_by_user_176 = item_names[purchased_by_user_176]
# visualizzo i primi 10 elementi
item_names_purchased_by_user_176[:10]

array(['Doctor Who - Robot [VHS]', 'Doctor Who - City of Death [VHS]',
       'Doctor Who: The Curse of Peladon [VHS]',
       'Doctor Who:  Time and The Rani [VHS]',
       'Doctor Who - Ghost Light [VHS]',
       'Doctor Who - The Visitation / Black Orchid [VHS]',
       'Doctor Who - Kinda [VHS]',
       'Doctor Who - The Mark of the Rani [VHS]',
       'Doctor Who - The Leisure Hive [VHS]',
       'Star Wars - Episode I, The Phantom Menace [VHS]'], dtype=object)

## Statistiche sugli Acquisti

- Gli array forniscono metodi per calcolare valori aggregati: somma, media, min/max, ecc.
- Ad esempio il metodo `sum` calcola la somma di tutti gli elementi di un array
- Possiamo utilizzarlo per ottenere il numero totale di acquisti analizzati

In [19]:
purchases.sum()

9683

- Oltre ad aggregare tutti i valori, possiamo compiere aggregazioni per righe (asse 0) o per colonne (asse 1)
- Applichiamo la somma per righe e per colonne alla matrice degli acquisti
  - `sum(1)` -> sommo tra loro le colonne -> ottengo il numero di prodotti acquistati per ogni utente
  - `sum(0)` -> sommo tra loro le righe -> ottengo il numero di utenti che hanno acquistato ciascun prodotto

In [20]:
users_purchases = purchases.sum(1)
items_purchases = purchases.sum(0)

- Con i metodi `min` e `max` possiamo estrarre i valori minimo e massimo
- Ad esempio, possiamo verificare che effettivamente ogni utente analizzato abbia almeno 30 acquisti

In [21]:
users_purchases.min()

30

- Tramite il metodo `mean`, possiamo estrarre il numero medio di acquisti per utente
- Con la funzione `median` possiamo estrarre il numero mediano

In [22]:
users_purchases.mean()

54.39887640449438

In [23]:
np.median(users_purchases)

45.0

- I metodi `argmin` e `argmax` restituiscono l'indice del valore minimo o massimo di un array piuttosto che il valore stesso
- Ad esempio, applicando `max` su `items_purchases` sappiamo quante volte è stato venduto il prodotto più acquistato

In [24]:
items_purchases.max()

50

- Con `argmax` vediamo quale sia il suo indice

In [25]:
items_purchases.argmax()

2091

- Dall'indice possiamo quindi risalire al nome

In [26]:
item_names[items_purchases.argmax()]

'The Sixth Sense [VHS]'

## Esercizi A: Estrarre Informazioni

- **1)** qual è il nome dell'utente che ha effettuato più acquisti?
- **2)** quali sono i nomi dei prodotti acquistati da almeno 35 utenti?
- **3)** quanti utenti hanno acquistato almeno 50 prodotti distinti?
  - suggerimento: ricordare che i booleani trattati come numeri sono 0 e 1, questo accade anche se vengono sommati

## Similarità tra Utenti

- Misuriamo la similarità tra due utenti come il numero di **prodotti distinti acquistati da entrambi**
- Rappresentando i dati in insiemi Python, abbiamo usato intersezioni tra gli insiemi di prodotti acquistati
- Avendo un vettore (riga della matrice) binario per ogni utente, il numero di prodotti in comune è dato dal **prodotto scalare**
  - il prodotto elemento per elemento è 1 solo dove in entrambi i vettori c'è 1, altrimenti è 0
  - il prodotto scalare è la somma di tali prodotti, ovvero il numero di 1
- Ad es., il numero di acquisti in comune tra il primo e il secondo utente è ...

In [27]:
# calcolo con funzione dot
np.dot(purchases[0], purchases[1])

4

In [28]:
# calcolo con operatore @ (richiede Python >= 3.5)
purchases[0] @ purchases[1]

4

### Matrice di Similarità

- Per ottenere i prodotti scalari tra tutte le righe, possiamo **moltiplicare la matrice per la sua trasposta**
  - si moltiplicano le righe della matrice originale per le colonne della trasposta, che corrispondono sempre alle righe dell'originale

In [29]:
common_purchases = np.dot(purchases, purchases.T)
#          oppure: purchases @ purchases.T

- Otteniamo una matrice quadrata, di ordine pari al numero di utenti

In [30]:
common_purchases.shape

(178, 178)

- Visualizziamo un estratto della matrice...

In [31]:
common_purchases[:10, :10]

array([[38,  4,  1,  0,  1,  2,  1,  1,  0,  2],
       [ 4, 32,  1,  0,  0,  2,  0,  1,  2,  4],
       [ 1,  1, 50,  1,  4,  2,  1,  2,  1,  7],
       [ 0,  0,  1, 41,  1,  1,  3,  1,  1,  2],
       [ 1,  0,  4,  1, 34,  1,  0,  2,  1,  2],
       [ 2,  2,  2,  1,  1, 47,  1,  0,  2,  1],
       [ 1,  0,  1,  3,  0,  1, 72,  1,  1,  2],
       [ 1,  1,  2,  1,  2,  0,  1, 40,  1,  6],
       [ 0,  2,  1,  1,  1,  2,  1,  1, 33,  3],
       [ 2,  4,  7,  2,  2,  1,  2,  6,  3, 51]], dtype=int32)

- ...possiamo ad esempio notare che
  - tra i primi due utenti ci sono 4 oggetti acquistati in comune
  - il primo ed il quarto utente non hanno alcun acquisto in comune

- Possiamo verificare che la matrice sia simmetrica controllando se è uguale alla sua trasposta
  - l'operatore `==` applicato tra matrici restituisce una matrice booleana con il confronto elemento per elemento
  - usiamo il metodo `all` per verificare che tutti i valori siano `True`

In [32]:
(common_purchases == common_purchases.T).all()

True

- In alternativa NumPy offre una funzione `array_equal` per verificare l'uguaglianza tra due array

In [33]:
np.array_equal(common_purchases, common_purchases.T)

True

### Diagonale della Matrice

- La diagonale della matrice creata contiene i prodotti scalari di ciascuna riga della matrice originale con se stessa
- Si può verificare che sono uguali al numero di acquisti per colonne calcolato sopra

In [34]:
np.array_equal(np.diag(common_purchases), users_purchases)

True

- Prima di procedere, **impostiamo a 0 tutti i valori sulla diagonale** per far sì che tra i simili di ciascun utente non sia incluso lui stesso
  - la funzione `fill_diagonal` imposta gli elementi della diagonale al valore dato

In [35]:
np.fill_diagonal(common_purchases, 0)
common_purchases[:5, :5]

array([[0, 4, 1, 0, 1],
       [4, 0, 1, 0, 0],
       [1, 1, 0, 1, 4],
       [0, 0, 1, 0, 1],
       [1, 0, 4, 1, 0]], dtype=int32)

_**Quesito:** qual è il massimo numero di prodotti in comune tra due utenti?_

## Stimare il Potenziale Interesse nei Prodotti

- Per stimare **quanto ciascun utente U sia potenzialmente interessato** in ciascun prodotto P, calcoliamo la somma delle similarità degli altri utenti che l'hanno acquistato
- Con Python abbiamo preso i punteggi di similarità tra U e altri utenti, filtrato quelli dei soli acquirenti di P e calcolato la loro somma
- Questa somma equivale in pratica al prodotto scalare tra:
  - la riga relativa a U di `common_purchases` che indica i punteggi di similarità con gli altri utenti e
  - la colonna relativa a P di `purchases` che indica quali utenti lo hanno acquistato
- Ad esempio, l'interesse stimato del 1° utente verso il 2° prodotto è:

In [36]:
common_purchases[0, :] @ purchases[:, 1]

5

- Per ottenere l'interesse di ogni utente U verso ogni prodotto P possiamo quindi usare come sopra il prodotto tra le due matrici
  - la matrice `common_purchases` ha forma **utenti** x utenti
  - la matrice `purchases` ha forma utenti x **prodotti**
  - la matrice risultante avrà forma **utenti x prodotti**

In [37]:
interest = common_purchases @ purchases

- Verifichiamo che la forma della matrice sia pari a quella della matrice degli acquisti (utenti x prodotti)

In [38]:
interest.shape == purchases.shape

True

- Visualizziamo un estratto della matrice...

In [39]:
interest[0:10, 0:10]

array([[ 0,  5,  0,  4,  4,  3,  1,  0,  4,  3],
       [ 3,  7,  1,  6,  1,  1,  2,  1,  2,  0],
       [ 6,  9,  0, 12,  0,  0,  1,  0,  9,  0],
       [ 1,  2,  2,  8,  0,  0,  0,  2,  5,  2],
       [ 3,  2,  0,  5,  0,  0,  0,  0,  2,  0],
       [ 2,  5,  0, 10,  0,  0,  1,  0,  5,  0],
       [ 0,  5,  5,  5,  1,  2,  1,  5,  2,  3],
       [ 1,  5,  0,  7,  0,  0,  0,  0,  0,  3],
       [ 2,  4,  0,  9,  0,  0,  1,  0,  4,  4],
       [ 6, 17,  0, 25,  0,  0,  0,  0,  9,  2]], dtype=int32)

- Ad esempio si ritiene che il 10° utente (ultima riga) sia molto interessato al 4° prodotto

### Scartare i Prodotti già Acquistati

- Nella matrice calcolata `interest`, sono ritenuti "interessanti" anche i prodotti **già acquistati** da ciascun utente
- Prima di proseguire, **azzeriamo i loro valori** in modo che siano considerati interessanti solo prodotti nuovi
- Convertiamo la matrice degli acquisti iniziale in forma booleana, per ottenere una "maschera"

In [40]:
purchases_mask = purchases.astype(np.bool)

- Utilizziamo quindi la maschera per **selezionare solo le posizioni corrispondenti** nella matrice dell'interesse all'acquisto e **impostarne i valori a 0**

In [41]:
interest[purchases_mask] = 0

## Ottenere _N_ Suggerimenti di Acquisto per ogni Utente

- Come la scorsa volta, vogliamo suggerire un numero fisso N di prodotti ad ogni utente, scegliendo quelli con interesse previsto maggiore

In [42]:
N = 20

- Per prima cosa assegniamo a ciascun prodotto un "ranking" in base all'interesse per ciascun utente
- Quindi selezioniamo per ogni cliente gli _N_ prodotti col ranking migliore

### Estrarre l'ordine dei valori: il metodo `argsort`

- Il metodo `argsort` su un vettore restituisce un vettore con i suoi **indici** (da 0 a N-1) **ordinati** secondo i valori

In [43]:
# sia dato il vettore...
x = np.array([32, 8, 2, 4, 16, 64, 1])
# ...il risultato di argsort è...
x.argsort()

array([6, 2, 3, 1, 4, 0, 5])

- Significa che l'elemento minimo è quello di indice 6 (1), seguito da quello di indice 2 (2) e così via fino al massimo di indice 5 (64)
- Nel caso di una matrice, l'operazione è eseguita **lungo una dimensione** a scelta (riga per riga o colonna per colonna)

In [44]:
np.array([[32,  8,  2,  4, 16, 64,  1],
          [ 8, 16,  4, 64, 32,  1,  2]]).argsort(1)   # 1 = riga per riga

array([[6, 2, 3, 1, 4, 0, 5],
       [5, 6, 2, 0, 1, 4, 3]])

### Calcolare il Ranking dei Prodotti

- Applicando l'operazione `argsort` due volte otteniamo il "ranking" dei dati, ovvero l'indice che ogni elemento avrebbe nell'array ordinato
  - Normalmente otteniamo quindi un array che associa 0 all'elemento più basso, 1 al secondo e così via

In [45]:
np.array([32, 8, 2, 4, 16, 64, 1]).argsort().argsort()

array([5, 3, 1, 2, 4, 6, 0])

- D'altra parte, invertendo l'array a cui è applicata l'operazione, otteniamo un array che associa 0 a partire dal valore più alto
- Applichiamo queste operazioni riga per riga all'array `interest`

In [46]:
interest_ranking = (-interest).argsort(1).argsort(1)

### Selezionare i Prodotti da Suggerire

- Abbiamo così ottenuto un array dove per ogni utente (riga) abbiamo i prodotti numerati univocamente da 0 a _N_-1 in ordine di interesse
- Da questo array possiamo quindi selezionare i **valori minori di _N_** per ottenere gli **_N_ prodotti di maggiore interesse** per ciascun utente
  - otteniamo una matrice di valori bool (True/False), che convertiamo in numerica con `astype`

In [47]:
suggestions = (interest_ranking < N).astype(np.int16)

- Questa è la **matrice dei prodotti suggeriti**, che associa ad ogni utente gli _N_ prodotti a cui è potenzialmente più interessato

## Accuratezza dei Suggerimenti di Acquisto

- Come la scorsa volta, valutiamo i suggerimenti ottenuti comparandoli con gli acquisti effettuati dopo l'analisi
- Tali acquisti sono riportati nel file `purchases-2014.csv`
- Come abbiamo fatto per il primo file, carichiamo le tuple ID utente-oggetto e riportiamole nella matrice
  - `zeros_like` crea una matrice di zeri di tipo e forma identiche a una data (esistono anche `empty_like` e `ones_like`)
  - scriviamo direttamente i dati nella matrice invece di caricarli in un insieme intermedio

In [48]:
purchases_updated = np.zeros_like(purchases)
with open("purchases-2014.csv", "r") as f:
    for uid, iid in csv.reader(f, delimiter=";"):
        purchases_updated[user_indices[int(uid)], item_indices[int(iid)]] = 1

### Selezionare solo i Nuovi Acquisti

- La nuova matrice riporta **tutti** gli acquisti, compresi quelli già indicati nella matrice precedente
- Vogliamo una matrice in cui siano riportati solo gli acquisti successivi all'analisi svolta sopra
- Possiamo ottenerla con una semplice differenza tra le due matrici

In [49]:
new_purchases = purchases_updated - purchases

- Quanti nuovi acquisti ha fatto mediamente ogni utente ?


In [50]:
mean_new_purchases = new_purchases.sum() / n_users
mean_new_purchases

32.98314606741573

### Quali Nuovi Acquisti sono stati Suggeriti?

- Abbiamo ora la matrice `suggestions` con gli acquisti _suggeriti_ e quella `new_purchases` con i nuovi acquisti _effettivi_
- Da queste possiamo individuare quali sono i suggerimenti **validi**, quelli a cui dopo l'analisi è corrisposto un acquisto
- Sono in pratica l'intersezione tra i due insiemi, che otteniamo moltiplicando le matrici elemento per elemento
  - ciascun valore della nuova matrice sarà 1 solo dove **entrambi** i valori delle due esistenti è 1

In [51]:
hits = suggestions * new_purchases

### Quanti Clienti hanno Ricevuto almeno un Suggerimento Valido?

- Possiamo usare la funzione `max` per vedere in quali righe di `hits` sia presente almeno un 1, corrispondente ad un suggerimento valido

In [52]:
satisfied_users = hits.max(1)

- Quanti sono gli utenti che hanno ricevuto un suggerimento valido?

In [53]:
satisfied_users.sum()

61

- Essendo un vettore binario, la media indica il rapporto di elementi 1 rispetto al totale

In [54]:
satisfied_users.mean()

0.34269662921348315

- Abbiamo quindi previsto per il **34,3%** degli utenti almeno un prodotto che poi hanno acquistato

## Esercizi B: Confronto con una Selezione Casuale di Prodotti

- Come nella scorsa esercitazione, per valutare quanto sia buono il risultato ottenuto, confrontiamolo con quello che si otterrebbe suggerendo prodotti a caso
- Utilizziamo la generazione casuale di array fornita da NumPy, inizializzando il relativo seed

In [55]:
np.random.seed(123)

- **1)** generare una matrice `random_interest` di forma pari ad `interest` con valori casuali
  - usare una funzione per generare valori casuali in un intervallo continuo, ad es `np.random.random`, indicando la forma desiderata

In [56]:
random_interest = ...

- **2)** impostare a 0 i valori della matrice corrispondenti a prodotti già acquistati

In [57]:
# ...

- **3)** selezionare gli _N_ prodotti di maggiore interesse per ogni utente
  - come sopra, generare la matrice con i ranking lungo ciascuna riga, quindi selezionare i ranking appropriati

In [58]:
random_interest_ranking = ...
random_suggestions = ...

- **4)** eseguire il confronto con i prodotti effettivamente acquistati ed estrarre il vettore che indica gli utenti con almeno un suggerimento valido

In [59]:
random_hits = ...
randomly_satisfied_users = ...

- **5)** calcolare la percentuale di tali utenti rispetto al totale

In [60]:
# ...